#TASK 4 Customer Segmentation Visualization

##Description:

- Visualize customer distribution by tenure
using pie or donut charts. (e.g., 0-12
months, 13-36 months, 37+ months).
Use a clustered bar chart to compare
average monthly charges across tenure
categories,
adding annotations to highlight significant
trends.

#STEP 1 : Importing

In [7]:
# ===================================================================
# TASK 4: CUSTOMER SEGMENTATION VISUALIZATION
# Industry Standard Implementation with Business Intelligence Focus
# ===================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import kaleido
warnings.filterwarnings('ignore')

# STEP 2 : CONFIGURATION



In [2]:
# === CONFIGURATION ===
class Config:
    COLORS_PRIMARY = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#4ECDC4', '#45B7D1']
    COLORS_TENURE = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    TENURE_BINS = [0, 12, 24, 48, 100]
    TENURE_LABELS = ['New (0-12m)', 'Growing (13-24m)', 'Mature (25-48m)', 'Loyal (49m+)']
    HIGH_VALUE_THRESHOLD = 70
    LOW_CHURN_THRESHOLD = 15

config = Config()

print("=" * 70)
print("TASK 4: CUSTOMER SEGMENTATION VISUALIZATION - EXECUTIVE DASHBOARD")
print("=" * 70)

TASK 4: CUSTOMER SEGMENTATION VISUALIZATION - EXECUTIVE DASHBOARD


# STEP 2 : LOAD DATA

In [3]:

# === LOAD DATA ===
try:
    df = pd.read_csv("Cleaned_Telco_Customer_Churn.csv")
    print("Data loaded from Cleaned_Telco_Customer_Churn.csv")
except FileNotFoundError:
    print("No dataset found. Generating sample data...")
    df = pd.DataFrame({
        'customerid': [f'ID-{i}' for i in range(1000)],
        'tenure': np.random.randint(1, 72, 1000),
        'monthlycharges': np.random.normal(65, 20, 1000),
        'churn': np.random.choice(['Yes', 'No'], 1000, p=[0.27, 0.73]),
        'contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], 1000),
        'paymentmethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], 1000)
    })

Data loaded from Cleaned_Telco_Customer_Churn.csv


# STEP 3 : SEGMENTATION





In [4]:
# === SEGMENTATION ===
df['tenure_segment'] = pd.cut(df['tenure'], bins=config.TENURE_BINS, labels=config.TENURE_LABELS, right=False)
df['value_segment'] = pd.qcut(df['monthlycharges'], q=4, labels=['Low Value', 'Medium Value', 'High Value', 'Premium'])

churn_rates = df.groupby('tenure_segment')['churn'].apply(lambda x: (x == 'Yes').mean() * 100)

def risk_category(segment):
    rate = churn_rates.get(segment, 0)
    if rate > 35:
        return 'High Risk'
    elif rate > 20:
        return 'Medium Risk'
    else:
        return 'Low Risk'

df['risk_segment'] = df['tenure_segment'].apply(risk_category)


# STEP 4 : Creating Dashboard

In [5]:

# === DASHBOARD ===
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Customer Distribution by Tenure", "Average Monthly Charges by Tenure",
                    "Churn Rate by Tenure", "Tenure vs Monthly Charges (Interactive)"),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Pie chart - Tenure distribution
tenure_counts = df['tenure_segment'].value_counts().reindex(config.TENURE_LABELS)
fig.add_trace(
    go.Pie(labels=tenure_counts.index, values=tenure_counts.values, hole=0.4),
    row=1, col=1
)

# Bar chart - Avg monthly charges
charges_df = df.groupby('tenure_segment')['monthlycharges'].mean().reindex(config.TENURE_LABELS)
fig.add_trace(
    go.Bar(x=charges_df.index, y=charges_df.values, marker_color=config.COLORS_PRIMARY[0]),
    row=1, col=2
)

# Bar chart - Churn rate
churn_df = churn_rates.reindex(config.TENURE_LABELS)
fig.add_trace(
    go.Bar(x=churn_df.index, y=churn_df.values, marker_color=config.COLORS_PRIMARY[1]),
    row=2, col=1
)

# Scatter plot - Tenure vs monthly charges
fig.add_trace(
    go.Scatter(
        x=df['tenure'], y=df['monthlycharges'],
        mode='markers',
        marker=dict(size=8, color=np.where(df['churn'] == 'Yes', 'red', 'green'), opacity=0.6),
        text=df['customerid']
    ),
    row=2, col=2
)

# Layout styling
fig.update_layout(
    title_text="Executive Dashboard: Customer Segmentation Analysis",
    height=900, width=1200,
    showlegend=True
)

fig.show()

In [9]:
#Saving dashborad
fig.write_html("task4_dashboard.html")